In [8]:
import cv2, os, pickle, uuid, time
import mysql.connector
from datetime import datetime
from deepface import DeepFace
from imgaug import augmenters as iaa
import numpy as np
from collections import Counter

db, session , course , lecture

load_encodings, retrain, #joined attendance # left attendance -> preprocess for recognition

recognize -> recognize

In [2]:
class Recognition:
    def __init__(self, db, semester_id, course_id,lecture_id,status):
        self.db = db
        self.semester_id = semester_id
        self.course_id = course_id
        self.lecture_id = lecture_id
        self.status = status
        self.confidence_threshold = 0.6
        self.load_encodings()
        self.predictions_buffer = []
        
    def load_encodings(self):
        if os.path.exists('known_encodings.pkl'):
            with open('known_encodings.pkl', 'rb') as f:
                self.results = pickle.load(f)
                self.known_encodings = [result[1] for result in self.results]
                self.file_paths = [result[0] for result in self.results]
        else:
            self.results = []
            self.known_encodings = []
            self.file_paths = []


    def retrain(self, augment=False, save_augments=False,n_samples=2):
        def augment_image(img):
            seq = iaa.Sequential([
                iaa.Affine(rotate=(-15, 15)),  # Random rotation
                iaa.Fliplr(0.5),  # Horizontal flip
                iaa.MultiplyBrightness((0.4, 1.5)),  # Random brightness
                iaa.GammaContrast((0.5, 1.5)),  # Gamma contrast
                iaa.Affine(scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}),  # Random scaling
                iaa.Affine(shear=(-7, 7))  # Random shear
            ])
            if augment:
                augmented_img = seq.augment_image(img)
                return augmented_img
            else:
                return img
    
        def process_folder(student_folder):
            student_id = os.path.basename(student_folder)
            print(student_id)
            image_files = [os.path.join(student_folder, file) for file in os.listdir(student_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
            encodings = []
    
            for img_path in image_files:
                img = cv2.imread(img_path)
    
                if augment:
                    augmented_images = [img]  # Include original image
                    for _ in range(n_samples):  # Generate additional samples
                        augmented_images.append(augment_image(img))
                else:
                    augmented_images = [img]
    
                for idx, augmented_img in enumerate(augmented_images):
                    encoding = DeepFace.represent(augmented_img, model_name='VGG-Face', enforce_detection=False)
                    if encoding:
                        encodings.append((img_path, encoding[0]['embedding']))
    
                        if save_augments and augment:  # Save augmented images if enabled
                            filename = os.path.splitext(os.path.basename(img_path))[0]
                            augmented_filename = f"augmented_{filename}_{idx}.jpg"
                            cv2.imwrite(os.path.join(student_folder, augmented_filename), augmented_img)
    
            return encodings
    
        db_folder = 'db'
        student_folders = [os.path.join(db_folder, folder) for folder in os.listdir(db_folder) if os.path.isdir(os.path.join(db_folder, folder))]
    
        self.results = []
        for student_folder in student_folders:
            self.results.extend(process_folder(student_folder))
    
        # Save computed embeddings and corresponding file paths
        with open('known_encodings.pkl', 'wb') as f:
            pickle.dump(self.results, f)
    
        self.load_encodings()


    def mark_attendance(self, student_id):
        # Marks attendance for a recognized student in the database
        try:
            cursor = self.db.cursor()
            # Fetch student's name from the database based on the student_id
            sql = "SELECT fullname FROM student_details WHERE student_id = %s"
            val = (student_id,)
            cursor.execute(sql, val)
            result = cursor.fetchone()
            if result:
                student_name = result[0]
            # Check if the student has already been marked present for the given course, session, and lecture
            sql = "SELECT * FROM attendance WHERE student_id = %s AND course_id = %s AND semester_id = %s AND lecture_id = %s"
            val = (student_id, self.course_id, self.semester_id, self.lecture_id)
            cursor.execute(sql, val)
            result = cursor.fetchone()
            if result:
                attendance_status = f"Attendance already taken for {student_name}"
                return
            else:
                sql = "INSERT INTO attendance (course_id, semester_id, lecture_id, timestamp, status, student_id) VALUES (%s, %s, %s, %s, %s, %s)"
                val = (self.course_id, self.semester_id, self.lecture_id, datetime.now(), self.status, student_id)
                cursor.execute(sql, val)
                self.db.commit()
                cursor.close()
        except mysql.connector.Error as err:
            print(f"Database error: {err}")

    def recognize(self, max_images=0, video_source=0, buffer_size=5):
        start_time = time.time()
        fps = 0
        frames = 0
        cap = cv2.VideoCapture(video_source)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        consecutive_detections = 0

        while True:
            ret, img = cap.read()
            if not ret:
                break
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            for (x, y, w, h) in faces:
                face_img = img[y:y + h, x:x + w]
                face_encoding = DeepFace.represent(face_img, model_name='VGG-Face', enforce_detection=False,
                                                   detector_backend='yolov8')
                if face_encoding:
                    face_encoding = face_encoding[0]['embedding']

                    best_match_id = None
                    confidence = 0.0

                    for j, known_encoding in enumerate(self.known_encodings):
                        if len(known_encoding) == len(face_encoding):
                            result = DeepFace.verify(known_encoding, face_encoding, model_name='VGG-Face', silent=True,
                                                     detector_backend='yolov8')
                            if result['verified'] and result['distance'] < self.confidence_threshold:
                                best_match_id = os.path.basename(os.path.dirname(self.file_paths[j]))
                                confidence = 1 - result['distance']
                                break

                    if best_match_id:
                        self.predictions_buffer.append(best_match_id)
                        consecutive_detections += 1

                        if consecutive_detections >= buffer_size:
                            most_common_prediction = Counter(self.predictions_buffer).most_common(1)[0][0]
                            cursor = self.db.cursor()
                            sql = "SELECT fullname FROM student_details WHERE student_id = %s"
                            val = (most_common_prediction,)
                            cursor.execute(sql, val)
                            result = cursor.fetchone()
                            if result:
                                student_name = result[0]
                            self.mark_attendance(most_common_prediction)
                            consecutive_detections = 0
                            self.predictions_buffer.clear()
            
                            cv2.putText(img, f"Recognized: {student_name}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(img, "Face Detected", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                else:
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    cv2.putText(img, "Face Not Recognized", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255),
                                2)
            frames += 1
            current_time = time.time()
            if current_time - start_time >= 1:
                fps = frames
                frames = 0
                start_time = current_time
            
            cv2.putText(img, f"FPS: {fps}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            cv2.imshow("Faces with Rectangles", img)

            if cv2.waitKey(1) == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

        


In [4]:
# 'joined', 'left'
semester_id = 2
course_id = 2
lecture_id = 1
status = 'joined'
# Example usage:
db_smp = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="e@123321xzX",
        port=3306,
        database="attendance"
    )

recognition = Recognition(db_smp, semester_id, course_id,lecture_id,status)

In [5]:
recognition.retrain()

1
2
3
4
5
6


In [7]:
recognition.recognize(25)

In [ ]:
# from fastapi import FastAPI, File, UploadFile
# from fastapi.responses import JSONResponse
# from fastapi.requests import Request
# from fastapi.encoders import jsonable_encoder
# from pydantic import BaseModel
# import cv2
# import numpy as np
# from deepface import DeepFace
# from imgaug import augmenters as iaa
# import mysql.connector
# from collections import Counter

# app = FastAPI()

# class Recognition(BaseModel):
#     semester_id: str
#     course_id: str
#     lecture_id: str
#     status: str

# @app.post("/recognize")
# async def recognize(Recognition: Recognition):
#     # Initialize the recognition model
#     recognition = Recognition(db, semester_id=Recognition.semester_id, course_id=Recognition.course_id, lecture_id=Recognition.lecture_id, status=Recognition.status)
#     recognition.load_encodings()
#     recognition.retrain()

#     # Start the recognition process
#     recognition.recognize()

#     return {"message": "Recognition process started"}

# @app.post("/mark_attendance")
# async def mark_attendance(student_id: str):
#     # Initialize the recognition model
#     recognition = Recognition(db, semester_id=semester_id, course_id=course_id, lecture_id=lecture_id, status=status)
#     recognition.mark_attendance(student_id)

#     return {"message": "Attendance marked successfully"}

# @app.post("/upload_images")
# async def upload_images(file: UploadFile = File(...)):
#     # Process the uploaded images
#     # ...

#     return {"message": "Images uploaded successfully"}

# if __name__ == "__main__":
#     uvicorn.run(app, host="0.0.0.0", port=8000)